# Sudoku

In [2]:
from z3 import *


X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]


cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]


rows_c   = [ Distinct(X[i]) for i in range(9) ]


cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ] 
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


# Šta Z3 ne može da uradi

Z3 deluje neverovatno moćno, i jeste. Međutim, pomoću Z3 rešavača možemo izraziti mnogo više stvari nego što on može da reši.

Na primer, faktorisanje celih brojeva je suština RSA kriptografije. Dok Z3 može faktorisati cele brojeve, ne može magično faktorisati čak ni umereno velike brojeve.

In [6]:
x,y = Ints("x y")
pubkey = 3	* 7
solve(x * y == pubkey, x > 1, y > 1) # easy peasy

[x = 7, y = 3]


In [7]:
x,y = Ints("x y")
pubkey = 1000000993	* 1000001011
solve(x * y == pubkey, x > 1, y > 1) # nope


failed to solve


Nelinearne jednačine će generalno biti teške. Nije nemoguće, ali teško. Z3 odmah odustaje ako pokušate da pronađete rešenje za naizgled rešiv problem koji uključuje eksponencijal. Z3 takođe ne može da razume sinuse, kosinuse i logaritme. Glavni izuzetak su ograničenja polinomske jednakosti, za koje z3 ima suštinsko razumevanje, međutim ove rutine mogu biti računski skupe.

In [ ]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print(s.check())


# Dokazati = Nema kontraprimera

Do sada smo koristili z3 da bismo pronašli rešenja za skup ograničenja/formula. Z3 je toliko efikasan u ovom procesu da može biti iscrpan i znati kada nema rešenja za pronalaženje. Ovo je korisno za dokazivanje teorema. Ako navedemo teoremu kao što je `Implies(And(p,k),p)` koja mora biti tačna za bilo koju vrednost p i k, možemo dokazati njenu istinitost tako što ćemo iscrpno tražiti kontraprimer i ne uspeti da ga pronađemo. Kontraprimer je dodeljivanje promenljivih za koje je interpretacija teoreme netačna, ili ekvivalentno za koje negacija Not(th) interpretira tačno.

solver.check() može da se vrati

 sat - "Našao sam rešenje. Možete ga tražiti pomoću solver.model()"
 nepoznato - Ovo znači "Odustajem". Možda negde postoji rešenje, a možda i ne.
 unsat - "ZNAM da nema rešenja. SVE sam probao".

Evo pojednostavljene (čak ni za toliko) verzije dokaza funkcije pogodnosti Z3 u kojoj kao dokaz izvodimo negaciju i interpretaciju nesatnih.


In [8]:
def prove2(f): # pojednostavljena verzija z3pi funkcije "dokaži"
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("proved")
    else:
        print("failed to prove")  

# Dokazivanje u iskaznoj logici

In [ ]:
p, q = Bools('p q')
print(And(p,q))
print(Or(p,q))
print(Xor(p,q))
print(Not(p))
print(Implies(p,q))
print(p == q)


my_true_thm = Implies(And(p,q), p)
prove(my_true_thm)


Za vežbu

Dokazati:

    De Morgan's Law p & q == ~ (~p | ~q)
    p -> q == ~ p | q
    Peirce's Law ((p -> q) -> p) -> p is always true



In [15]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
print(demorgan)

def prove(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        print("proved")
    else:
        print("failed to prove")

print("Proving demorgan...")
prove(demorgan)



And(p, q) == Not(Or(Not(p), Not(q)))
Proving demorgan...
proved
